This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [9]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'sXkbi3RPJZZexc1SyQdu'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules
import requests

In [11]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/WIKI/FB.json?ollapse=annual&rows=6&start_date=2014-01-01&end_date=2014-12-31&api_key='+API_KEY
r = requests.get(url)
r.json()

{'dataset': {'id': 9775687,
  'dataset_code': 'FB',
  'database_code': 'WIKI',
  'name': 'Facebook Inc. (FB) Prices, Dividends, Splits and Trading Volume',
  'description': "End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Facebook, Inc. (FB). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (www.crsp.com/products/documentation/crsp-calculations)\n\nThis data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.\n\nThis data is part of Quandl's Wiki initiative to get financial data permanently into the public domain. Quandl relies on users like you to flag errors and provide data where data is wrong or missing. Get involved: connect@quandl.com\n",
  'refreshed_at': '2018-03-27T21:46:11.036Z',
  'newest_available_date': '2018-03-27',
  'old

In [24]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()['dataset']

In [13]:
r.json()['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [14]:
START_END = '2017-01-01'
END_DATE = '=2017-12-31'
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date='+START_END+'&end_date='+END_DATE+'&api_key='+API_KEY
response = requests.get(url)

2. Convert the returned JSON object into a Python dictionary.

In [15]:
if (response.status_code == 200):
    stock_dict =  response.json()
else:
    print("Error fetching data")

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [16]:
column_names = stock_dict['dataset']['column_names']
#print(column_names)

In [17]:
data_list = stock_dict['dataset']['data']
#print(data_list)

In [18]:
# Dictionary -  keys: columns, values:list of values on all days
required_cols = ['Open', 'High','Low','Close','Traded Volume']
formatted_stock_dict = {}
for data in data_list:
    for col in required_cols:
        index_in_data = column_names.index(col)
        formatted_stock_dict.setdefault(col, []).append(data[index_in_data])
#print(formatted_stock_dict)

In [19]:
opening_prices_sorted = list(filter(None,formatted_stock_dict['Open']))
opening_prices_sorted.sort()
print("The highest opening price:" , opening_prices_sorted[-1])
print("The lowest opening price:" , opening_prices_sorted[0])

The highest opening price: 53.11
The lowest opening price: 34.0


4. What was the largest change in any one day (based on High and Low price)?

In [20]:
high_low_diff_per_day = [formatted_stock_dict['High'][i] - formatted_stock_dict['Low'][i] for i in range(len(formatted_stock_dict['High']))]
high_low_diff_per_day.sort()
print("The largest change in any one day:",round(high_low_diff_per_day[-1],4))

The largest change in any one day: 2.81


5. What was the largest change between any two days (based on Closing Price)?


In [21]:
close_diff = []
for i in range(len(formatted_stock_dict['Close'])-1):
    j = i+1
    diff = abs(formatted_stock_dict['Close'][i] - formatted_stock_dict['Close'][j])
    close_diff.append(diff)
close_diff.sort()
print("The largest change between any 2 days:", round(close_diff[-1],4))

The largest change between any 2 days: 2.56


6. What was the average daily trading volume during this year?

In [22]:
avg_daily_traded_volume = sum(formatted_stock_dict['Traded Volume'])/len(formatted_stock_dict['Traded Volume'])
print("The average trading volume:", round(avg_daily_traded_volume,4))


The average trading volume: 89124.3373


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [23]:
def median(list_to_sort):
    if(len(list_to_sort)) > 1:
        list_to_sort.sort()
        mid_value = int(str(len(list_to_sort)/2).split(".")[0])
        if len(list_to_sort)%2 != 0:
            return list_to_sort[mid_value]
        else:
            second_index = mid_value
            first_index = second_index-1
            return (list_to_sort[first_index] + list_to_sort[second_index])/2
    else:
        print("Median does not exist")
        
median_trading_volume = median(formatted_stock_dict['Traded Volume'])
print("The median trading volume:",median_trading_volume)

The median trading volume: 76286.0
